In [ ]:
!pip install pytrends

In [ ]:
import pandas as pd
from pytrends.request import TrendReq

In [ ]:
#可不做 链接google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pandas==1.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 41.4 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.2
    Uninstalling pandas-2.1.2:
      Successfully uninstalled pandas-2.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


使用pytrend抓取各个州的谷歌搜索兴趣

In [ ]:
from pytrends.request import TrendReq
import pandas as pd
import random
import time
import os

# 美国50个州的缩写
us_states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
             "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
             "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
             "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
             "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

def download_interest_data(region, keyword, start_date, end_date):
    # 创建 pytrends 对象
    pytrends = TrendReq(hl='en-US', tz=360)

    # 设置搜索关键字和时间范围
    kw_list = [keyword]
    timeframe = start_date + ' ' + end_date

    try:
        # 获取搜索兴趣数据
        pytrends.build_payload(kw_list, geo=region, timeframe=timeframe)
        interest_data = pytrends.interest_over_time()

        # 将日期作为索引
        interest_data.index = pd.to_datetime(interest_data.index)

        # 创建关键词目录
        keyword_dir = os.path.join(keyword)
        if not os.path.exists(keyword_dir):
            os.makedirs(keyword_dir)

        # 生成文件名
        file_name = f"{region}_{keyword}.csv"
        file_path = os.path.join(keyword_dir, file_name)

        # 保存为 CSV 文件
        interest_data.to_csv(file_path)

        state_abbreviation = region.replace("US-", "")  # 去除 "US-" 前缀
        us_states.remove(state_abbreviation)
        print("finish downloading and delete", region)

    except Exception as e:
        if "429" in str(e):
            print(f"Got 429 error. Sleeping for 1 minute...")
            time.sleep(60)
            print(f"Retrying {region}...")
            download_interest_data(region, keyword, start_date, end_date)
        else:
            print(f"Error downloading data for {region}: {str(e)}")


In [ ]:
# 要下载的关键词和时间范围
keywords = ["mental health"]
start_date = "2019-01-01"
end_date = "2023-10-31"

# 逐个关键词生成搜索兴趣数据
for keyword in keywords:
    while us_states:
        state = random.choice(us_states)
        download_interest_data("US-" + state, keyword, start_date, end_date)
        print("Finish", state, keyword)
        sleeptime = random.randint(5, 10)
        print("Sleep for", sleeptime, "seconds")
        time.sleep(sleeptime)

print("All data downloaded.")

finish downloading and delete US-CO
Finish CO mental health
Sleep for 6 seconds
finish downloading and delete US-MS
Finish MS mental health
Sleep for 5 seconds
finish downloading and delete US-AR
Finish AR mental health
Sleep for 9 seconds
finish downloading and delete US-MT
Finish MT mental health
Sleep for 10 seconds
finish downloading and delete US-MO
Finish MO mental health
Sleep for 6 seconds
finish downloading and delete US-VA
Finish VA mental health
Sleep for 6 seconds
finish downloading and delete US-NV
Finish NV mental health
Sleep for 6 seconds
finish downloading and delete US-HI
Finish HI mental health
Sleep for 5 seconds
finish downloading and delete US-IN
Finish IN mental health
Sleep for 8 seconds
finish downloading and delete US-FL
Finish FL mental health
Sleep for 6 seconds
Got 429 error. Sleeping for 1 minute...
Retrying US-OH...
finish downloading and delete US-OH
Finish OH mental health
Sleep for 5 seconds
finish downloading and delete US-PA
Finish PA mental health
S

In [ ]:
!ls mental\ health/


'US-AK_mental health.csv'  'US-LA_mental health.csv'  'US-OH_mental health.csv'
'US-AL_mental health.csv'  'US-MA_mental health.csv'  'US-OK_mental health.csv'
'US-AR_mental health.csv'  'US-MD_mental health.csv'  'US-OR_mental health.csv'
'US-AZ_mental health.csv'  'US-ME_mental health.csv'  'US-PA_mental health.csv'
'US-CA_mental health.csv'  'US-MI_mental health.csv'  'US-RI_mental health.csv'
'US-CO_mental health.csv'  'US-MN_mental health.csv'  'US-SC_mental health.csv'
'US-CT_mental health.csv'  'US-MO_mental health.csv'  'US-SD_mental health.csv'
'US-DE_mental health.csv'  'US-MS_mental health.csv'  'US-TN_mental health.csv'
'US-FL_mental health.csv'  'US-MT_mental health.csv'  'US-TX_mental health.csv'
'US-GA_mental health.csv'  'US-NC_mental health.csv'  'US-UT_mental health.csv'
'US-HI_mental health.csv'  'US-ND_mental health.csv'  'US-VA_mental health.csv'
'US-IA_mental health.csv'  'US-NE_mental health.csv'  'US-VT_mental health.csv'
'US-ID_mental health.csv'  'US-NH_mental

下载各州数据

In [ ]:
import zipfile
import os

# 指定要压缩的文件夹路径
folder_to_compress = "mental health/"  # 这里请替换为您的文件夹路径

# 指定要保存压缩文件的名称
zip_file_name = "mental_data.zip"

# 使用zipfile库创建一个zip文件
with zipfile.ZipFile(zip_file_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # 遍历文件夹中的所有文件并将它们添加到zip文件中
    for root, dirs, files in os.walk(folder_to_compress):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, folder_to_compress)
            zipf.write(file_path, arcname)

# 下载生成的zip文件
from google.colab import files
files.download(zip_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

合并50个州的搜索兴趣为一个csv

In [ ]:
import os
import pandas as pd

# 指定包含所有各州数据的文件夹路径
folder_path = "mental health/"  # 请替换为您的文件夹路径

# 获取文件夹中的所有文件名
file_names = os.listdir('mental health/')

# 创建一个空的DataFrame用于存储合并后的数据
merged_data = pd.DataFrame()

# 遍历每个文件并合并数据
for file_name in file_names:
    # 从文件名中提取州简称，假设文件名的格式是"州简称_depression.csv"
    state_code = file_name.split("_")[0]

    # 读取州的数据文件并添加到合并后的DataFrame中，同时删除isPartial列
    state_data = pd.read_csv(os.path.join(folder_path, file_name))

    # 检查是否存在'depression'列，如果存在，则添加到合并后的DataFrame中
    if 'mental health' in state_data.columns:
        merged_data[state_code] = state_data['mental health']
    else:
      print (file_name)

# 使用日期作为索引
merged_data.index = pd.to_datetime(state_data['date'])

# 保存合并后的数据为CSV文件
merged_data.to_csv('state_mental.csv')


US-OH_mental health.csv


In [ ]:
print(merged_data.head())


            US-AR  US-IA  US-LA  US-WY  US-AZ  US-FL  US-MA  US-WA  US-TN  \
date                                                                        
2019-01-06     52     66      0     31      0      0     52     63     79   
2019-01-13     58     56      0     27     46      0     54     61     80   
2019-01-20     46     49      0     19      0      0     55     53     77   
2019-01-27     57     50      0     70      0      0     62     61     84   
2019-02-03     67     60      0     28      0      0     58     52     71   

            US-NH  ...  US-MT  US-WV  US-ID  US-KY  US-PA  US-IL  US-TX  \
date               ...                                                    
2019-01-06     57  ...      0     34     58     50      0      0      0   
2019-01-13     56  ...      0     43     48     49      0      0      0   
2019-01-20     54  ...      0     38     59     48      0      0      0   
2019-01-27     68  ...      0     14     31     52      0      0      0   
2019-02-03

In [ ]:
files.download('state_mental.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>